In [ ]:
import cv2
import numpy as np
from numpy.random import rand
import tensorflow as tf
import sys
import matplotlib.pyplot as plt

def createTransform(scale=1, angle=0, x=0, y=0, p1=0, p2=0):
    angle = angle/180*np.pi
    return np.array([[scale*np.cos(angle), -scale*np.sin(angle), x ],
                    [scale*np.sin(angle),  scale*np.cos(angle),  y ],
                    [p1, p2, 1]])

def random_perspective_placing(fg, bg, border_width):
    bh, bw, _ = bg.shape
    h, w, _ = fg.shape
    dx, dy = np.random.normal(0, 1.2/3, size=2)
    p1, p2 = np.random.normal(0, 0.3/3, size=2)
    a = np.random.rand()*360
    s = np.random.normal(0.4, 0.12 /3)
    bx = np.random.normal(0, bw/3*0.1)
    by = np.random.normal(0, bh/3*0.1)

    DocCent = createTransform(x=-w/2, y=-h/2)
    AS = createTransform(angle=a, scale=2/h)
    T = createTransform(x=dx, y=dy)
    P = createTransform(p1=p1, p2=p2)
    S = createTransform(scale=bh*s)
    BGCent = createTransform(x=bw/2+bx, y=bh/2+by)
    perspective_trans = BGCent@S@P@T@AS@DocCent

    input = cv2.warpPerspective(fg, perspective_trans, (bw, bh), bg, borderMode=cv2.BORDER_TRANSPARENT) 
    
    black = np.zeros((bh, bw))
    pts = np.array([[0,0,1], [w,0,1], [w,h,1], [0,h,1]]).T
    pts = perspective_trans @ pts
    pts = pts[:2, :] / pts[2, :]
    pts = np.int32(pts.T)
    output = cv2.polylines(black, [pts], isClosed=True, color=255, thickness=border_width)

    return input, output.astype(np.uint8)

def random_cut(img, w, h):
    ih, iw, _ = img.shape
    r1 = int(rand() * (ih-h))
    r2 = r1 + h
    c1 = int(rand() * (iw-w))
    c2 = c1 + w
    return img[r1:r2, c1:c2]

def generate_edge_data(w, h, border_width):
    bg_scale = 0.4
    docName = np.floor(rand()*25)
    bgName = np.floor(rand()*397)
    doc = cv2.imread('./raw_dataset/docs/{name:.0f}.jpg'.format(name=docName))
    bg = cv2.imread('./raw_dataset/backgrounds/{name:.0f}.jpg'.format(name=bgName))
    bg = cv2.resize(bg, None, fx=bg_scale, fy=bg_scale)
    bg = random_cut(bg, w, h)
    return random_perspective_placing(doc, bg, border_width)

def test(w, h):
    for i in range(10):
        input, output = generate_edge_data(w, h)
        plt.imshow(input)
        plt.show()
        plt.imshow(output, cmap='gray')
        plt.show()

def generate(w, h, start, numb, border_width=1): 
    for i in range(start, start+numb):
        input, output = generate_edge_data(w, h, border_width)
        cv2.imwrite("./dataset/x/{i}.jpg".format(i=i), input)
        cv2.imwrite("./dataset/y/{i}.jpg".format(i=i), output)

        sys.stdout.write("\r" + 'Generating Images: {i}.jpg'.format(i=i))
        sys.stdout.flush()
    print('finished')

def loadData(split_rate=0.8):
    def parse_files(x_fname, y_fname):
        image_string = tf.io.read_file(x_fname)
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        x = tf.cast(image_decoded, tf.float32) / 127.5 - 1

        image_string = tf.io.read_file(y_fname)
        image_decoded = tf.image.decode_jpeg(image_string, channels=1)
        y = tf.cast(image_decoded, tf.float32) / 255.0

        return x, y

    x = tf.data.Dataset.list_files('./dataset/x/*.jpg', shuffle=False)
    x = list(x.as_numpy_iterator())

    y = tf.data.Dataset.list_files('./dataset/y/*.jpg', shuffle=False)
    y = list(y.as_numpy_iterator())

    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.shuffle(buffer_size=len(x), seed=123).map(parse_files)
    print('data size: ' + str(len(x)))

    spliter = int(len(x) * split_rate)
    ds_train = ds.take(spliter)
    ds_test = ds.skip(spliter)
    return ds_train, ds_test

# test(w=512, h=512)
# generate(w=512, h=512, start=0, numb=1)
# read_dataset(numb=3000)


0 (None, 512, 512, 3) input_1
1 (None, 256, 256, 32) Conv1
2 (None, 256, 256, 32) bn_Conv1
3 (None, 256, 256, 32) Conv1_relu
4 (None, 256, 256, 32) expanded_conv_depthwise
5 (None, 256, 256, 32) expanded_conv_depthwise_BN
6 (None, 256, 256, 32) expanded_conv_depthwise_relu
7 (None, 256, 256, 16) expanded_conv_project
8 (None, 256, 256, 16) expanded_conv_project_BN
9 (None, 256, 256, 96) block_1_expand
10 (None, 256, 256, 96) block_1_expand_BN
11 (None, 256, 256, 96) block_1_expand_relu
12 (None, 257, 257, 96) block_1_pad
13 (None, 128, 128, 96) block_1_depthwise
14 (None, 128, 128, 96) block_1_depthwise_BN
15 (None, 128, 128, 96) block_1_depthwise_relu
16 (None, 128, 128, 24) block_1_project
17 (None, 128, 128, 24) block_1_project_BN
18 (None, 128, 128, 144) block_2_expand
19 (None, 128, 128, 144) block_2_expand_BN
20 (None, 128, 128, 144) block_2_expand_relu
21 (None, 128, 128, 144) block_2_depthwise
22 (None, 128, 128, 144) block_2_depthwise_BN
23 (None, 128, 128, 144) block_2_depthw